In [11]:
!pip install datasets
!pip install -q pytorch-lightning nltk
!pip install sentencepiece
!pip uninstall -y ninja && pip install ninja
!pip install wandb
!pip install accelerate
!pip install OmegaConf


In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [5]:

# Check PyTorch version and GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"CUDNN version: {torch.backends.cudnn.version() if torch.cuda.is_available() else 'N/A'}")

PyTorch version: 2.1.0+cu121
CUDA available: True
CUDA version: 12.1
CUDNN version: 8902


In [6]:
!ninja --version

1.11.1.git.kitware.jobserver-1


In [7]:
!echo $?

0


In [ ]:
# !pip install flash-attn==1.0.9 --no-build-isolation
# !unzip /content/flash-attention-1.0.9.zip -d /content/flash_attention
# !pip install /content/flash_attention/flash-attention-main/csrc/xentropy
# !pip install /content/flash_attention/flash-attention-main/csrc/rotary
# !pip install /content/flash_attention/flash-attention-main/csrc/layer_norm

In [ ]:
!pip install flash-attn --no-build-isolation

In [ ]:
!unzip /content/flash-attention-main.zip -d /content/flash_attention

In [13]:
!pip install /content/flash_attention/flash-attention-main/csrc/fused_dense_lib

Processing ./flash_attention/flash-attention-main/csrc/fused_dense_lib
  Preparing metadata (setup.py) ... done
  Created wheel for fused-dense-lib: filename=fused_dense_lib-0.0.0-cp310-cp310-linux_x86_64.whl size=2700836 sha256=f50da5b76fc5c33ff7eb77efc26f5e96cdbf8351f0dcc8501acfe8955bb467e6
  Stored in directory: /tmp/pip-ephem-wheel-cache-rkf03uxw/wheels/74/55/5d/f379341f8396d4406dcec27c097484ae9ea079a3c5b28feec9
Successfully built fused-dense-lib


In [10]:
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from flash_attn.models.gpt import GPTLMHeadModel
import torch
seqlen = 512
hidden_dim = 2048
nheads = 16
n_layer = 24
rotary_emb_fraction = 0.5
config = GPT2Config(vocab_size=50257, n_positions=seqlen, n_embd=hidden_dim,
                    n_layer=n_layer, n_head=nheads,
                    scale_attn_by_inverse_layer_idx=True,
                    rotary_emb_fraction=rotary_emb_fraction,
                    use_flash_attn=True, fused_mlp=True,
                    fused_bias_fc=True, fused_dropout_add_ln=True,
                    pad_vocab_size_multiple=8,torch_dtype=torch.float16)
model = GPTLMHeadModel(config)

In [11]:
print(model)

GPTLMHeadModel(
  (transformer): GPTModel(
    (embeddings): GPT2Embeddings(
      (word_embeddings): Embedding(50264, 2048)
      (position_embeddings): Embedding(512, 2048)
    )
    (layers): ModuleList(
      (0-23): 24 x Block(
        (mixer): MHA(
          (rotary_emb): RotaryEmbedding()
          (Wqkv): FusedDense(in_features=2048, out_features=6144, bias=True)
          (inner_attn): FlashSelfAttention(
            (drop): Dropout(p=0.1, inplace=False)
          )
          (inner_cross_attn): FlashCrossAttention(
            (drop): Dropout(p=0.1, inplace=False)
          )
          (out_proj): FusedDense(in_features=2048, out_features=2048, bias=True)
        )
        (dropout1): Dropout(p=0.1, inplace=False)
        (drop_path1): StochasticDepth(p=0.0, mode=row)
        (norm1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): FusedMLP(
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, 

In [12]:
import torch
import transformers
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader, Dataset
from transformers import AutoTokenizer
from datasets import load_dataset
from pytorch_lightning import LightningDataModule

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning import Callback
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks import LearningRateMonitor

import pytorch_lightning as pl

import wandb

In [4]:
from datasets import load_dataset

data =load_dataset("wikimedia/wikisource", "20231201.en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/208279 [00:00<?, ? examples/s]

In [5]:
print(data)

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 208279
    })
})


In [15]:
import torch
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer
from torch.utils.data import DataLoader, Dataset, random_split
tokenizer =  GPT2Tokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
class Wikidata(Dataset):
    def __init__(self, dataset, max_length=512, tokenizer=tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = self.dataset[idx]["text"]

        inputs = self.tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
        )

        input_ids = inputs["input_ids"].squeeze()

        labels = input_ids.clone()
        labels[:-1] = input_ids[1:]
        labels[-1] = self.tokenizer.eos_token_id

        return {
            "input_ids": input_ids,
            "labels": labels
        }


In [16]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader
from torch.nn import functional as F

train_dataset = Wikidata(data["train"])

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=5000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,

)


In [17]:
trainer.train()

AssertionError: libcuda.so cannot found!


Follow thread https://github.com/pytorch/pytorch/issues/107960 nishaanthkanna solution

In [18]:
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link



In [19]:
trainer.train()

AssertionError: 

In [26]:
tokenizer =  GPT2Tokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
class Wikidata(Dataset):
    def __init__(self, dataset, max_length=512, tokenizer=tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = self.dataset[idx]["text"]

        inputs = self.tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
        )
        # change the input to float 16
        input_ids = inputs["input_ids"].squeeze().to(torch.float16)

        labels = input_ids.clone()
        labels[:-1] = input_ids[1:]
        labels[-1] = self.tokenizer.eos_token_id

        return {
            "input_ids": input_ids,
            "labels": labels
        }


In [27]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader
from torch.nn import functional as F

train_dataset = Wikidata(data["train"])

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=5000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,

)


In [28]:
trainer.train()

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.HalfTensor instead (while checking arguments for embedding)

#### Thử lại xem prepare dataset có vấn đề không

In [29]:
datasets=data
datasets = datasets['train'].train_test_split(test_size=0.005)
data_train = datasets['test']
model_checkpoint = "gpt2"
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

In [30]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512, padding="max_length")
tokenized_datasets = data_train.map(tokenize_function, batched=True, num_proc=2, remove_columns=["text",'id','url','title'])


Map (num_proc=2):   0%|          | 0/1042 [00:00<?, ? examples/s]

In [40]:
def collate_fn(batch):
    input_ids = [item["input_ids"] for item in batch]
    attention_masks = [item["attention_mask"] for item in batch]
    labels = [item["input_ids"] for item in batch]

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    labels = torch.tensor(labels)

    # Pad sequences to the same length
    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True)
    attention_masks = torch.nn.utils.rnn.pad_sequence(attention_masks, batch_first=True)
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "labels": labels,
    }

# Create DataLoader using tokenized_datasets['train']
train_dataloader = DataLoader(tokenized_datasets, batch_size=4, shuffle=True, collate_fn=collate_fn)


# Set the device to GPU
device = torch.device("cuda")
model.cuda()

# Set up Trainer
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=5000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataloader.dataset,
)

trainer.train()


AssertionError: 

In [ ]:
# Save the model after training
trainer.save_model("./gpt2-finetuned")